In [ ]:
import pandas as pd
import os
import sys
from datetime import datetime
from dateutil.relativedelta import relativedelta 
import mydashtools as dt
import plotly.express as px
pd.options.plotting.backend = "plotly"
if dt.is_notebook():
    __file__ = "notebook/"

events = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../events/barco_events_all.csv", sep=";",decimal=',',low_memory=False, index_col=False)

limit_year = None
if os.getenv("LIMIT_YEAR") is not None:
    limit_year = os.getenv("LIMIT_YEAR")
if limit_year:
    events = events[events['date'] <= limit_year+'-12-31']

events['date'] = pd.to_datetime(events['date'])
events.set_index('date', inplace=True)
events_by_day = events.resample('D')['titre'].apply(' / '.join).reset_index()
events_by_day['date'] = events_by_day['date'].apply(lambda x: str(x)[0:10])
events_by_day.set_index('date', inplace=True)
d2e = events_by_day[events_by_day['titre'] != '']['titre'].to_dict()

if dt.is_notebook():
    print(events.head())

if events.index.empty:
    sys.exit()

In [ ]:
if limit_year:
    now = max(events.index)
    limit_date = now + relativedelta(years=-1)
else:
    now = datetime.now()
    limit_date = now + relativedelta(months=-1)

events_par_semaine = events[events.index > str(limit_date)[0:19]]
events_par_semaine['titre'] = events_par_semaine['titre'].apply(lambda x: x[:50])
events_par_semaine = events_par_semaine.resample('W')[['url', 'titre']].aggregate({'url': 'count', 'titre': " | ".join}).rename(columns={'url': 'Nb évènements'})

events_par_semaine["Nb d'évènements"] = events_par_semaine[events_par_semaine.index <= now.isoformat()]["Nb évènements"]
events_par_semaine["Nb d'évènements futurs"] = events_par_semaine[events_par_semaine.index > now.isoformat()]["Nb évènements"]
events_par_semaine['titre'] = events_par_semaine['titre'].apply(lambda x: x[:125])

del events_par_semaine["Nb évènements"]
fig = px.bar(events_par_semaine,
              y=['Nb d\'évènements', 'Nb d\'évènements futurs'], hover_data=['titre'],
             color_discrete_map={"Nb d'évènements": '#636efa', "Nb d'évènements futurs": '#9aa1fa'}
            )
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
fig.update_yaxes(dtick="n")

dt.fig_save_or_show(fig, 'evenements_01_events_par_semaine', limit_year)

In [ ]:
events_mensuels_par_an = events.resample('MS')[['url']].count().rename(columns={'url': 'Nb d\'évènements'})

fig = px.bar(events_mensuels_par_an)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'evenements_02_events_mensuels_par_an', limit_year)

In [ ]:
events_par_an = events.resample('YS')[['url']].count().rename(columns={'url': 'Nb d\'évènements'})
events_par_an.reset_index(inplace=True)
events_par_an['Année'] = events_par_an['date'].apply(lambda x: str(x)[0:4])
events_par_an.set_index('Année', inplace=True)
del events_par_an['date']

fig = px.bar(events_par_an)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'evenements_03_events_par_an', limit_year)

In [ ]:
zelty = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../zelty/zelty.csv", sep=";",decimal=',',low_memory=False, index_col=False)
zelty['date'] = pd.to_datetime(zelty['date']).apply(lambda x: str(x)[0:10])
if (limit_year):
    zelty = zelty[zelty['date'] <= limit_year+'-12-31']
zelty_by_day = zelty.groupby('date')[['price']].sum()
zelty_by_day['evenement'] = zelty_by_day.index.map(d2e)
#zelty_by_day.sort_values('price', ascending=False).dropna()


In [ ]:
adhésions = zelty[zelty['category'] == 'Adhésion'].groupby('date')[['price']].count().rename(columns={'price': 'adhésions'})
adhésions['evenement'] = adhésions.index.map(d2e)
zelty_by_day['adhésions'] = adhésions['adhésions']
zelty_by_day['jour'] = pd.to_datetime(zelty_by_day.index).strftime('%A') 
evenement_adhesion_prix = zelty_by_day.dropna().sort_index(ascending=False)[['jour','evenement', 'adhésions', 'price']]
dt.true_or_html_to_file(evenement_adhesion_prix, 'evenements_04_adhesion_prix', limit_year) and evenement_adhesion_prix